Naive Bayes

importing

In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

datasets

In [56]:
fake = pd.read_csv('/content/news/Fake.csv', delimiter = ',')
true = pd.read_csv('/content/news/True.csv', delimiter = ',')


combining datasets to one, and adding a column to define fake/real

In [57]:
fake['sentiment']= 0
true['sentiment']= 1

dataset =pd.DataFrame()
dataset = pd.concat([true, fake])
dataset.count

<bound method DataFrame.count of                                                    title  \
0      As U.S. budget fight looms, Republicans flip t...   
1      U.S. military to accept transgender recruits o...   
2      Senior U.S. Republican senator: 'Let Mr. Muell...   
3      FBI Russia probe helped by Australian diplomat...   
4      Trump wants Postal Service to charge 'much mor...   
...                                                  ...   
23476  McPain: John McCain Furious That Iran Treated ...   
23477  JUSTICE? Yahoo Settles E-mail Privacy Class-ac...   
23478  Sunnistan: US and Allied ‘Safe Zone’ Plan to T...   
23479  How to Blow $700 Million: Al Jazeera America F...   
23480  10 U.S. Navy Sailors Held by Iranian Military ...   

                                                    text       subject  \
0      WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1      WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2      WASHINGTON (Reute

cleaning

In [58]:
column = ['date','subject']
dataset = dataset.drop(columns=column)
input_array=np.array(dataset['title'])

In [59]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, 40000):
    review = re.sub('[^a-zA-Z]', ' ', input_array[i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


creating bag of world model

In [60]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[0:40000, 2].values


splitting data to training and test set

In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

calculating percentages from the confusion matrices

In [62]:
def calculate_accuracy(confusion_matrix):
    # Extract the values from the confusion matrix
    true_positives = confusion_matrix[0][0]
    true_negatives = confusion_matrix[1][1]
    false_positives = confusion_matrix[0][1]
    false_negatives = confusion_matrix[1][0]

    # Calculate the accuracy
    accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives)

    # Convert accuracy to percentage
    accuracy_percentage = accuracy * 100

    return accuracy_percentage


fitting to Naive Bayes

In [63]:
import time
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix

# Start the timer
start_time = time.time()

# Fitting Naive Bayes to the Training set
classifier = GaussianNB()
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
cm_naivebayes = confusion_matrix(y_test, y_pred)

# Stop the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print('Naive Bayes confusion matrix:')
print(cm_naivebayes)
accuracy_naivebayes = calculate_accuracy(cm_naivebayes)
print('Accuracy of Naive Bayes method:', accuracy_naivebayes)
print('Elapsed time:', elapsed_time, 'seconds')


Naive Bayes confusion matrix:
[[3096  656]
 [ 280 3968]]
Accuracy of Naive Bayes method: 88.3
Elapsed time: 3.3326354026794434 seconds


fitting to Random Forest

In [69]:
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

# Start the timer
start_time = time.time()

# Fitting Random Forest to the Training set
classifier3 = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)
classifier3.fit(X_train, y_train)

# Predicting the Test set results
y_predR = classifier3.predict(X_test)

# Making the Confusion Matrix
cm_randomforest = confusion_matrix(y_test, y_predR)

# Stop the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print('Random Forest confusion matrix:')
print(cm_randomforest)
accuracy_randomforest = calculate_accuracy(cm_randomforest)
print('Accuracy of Random Forest method:', accuracy_randomforest)
print('Elapsed time:', elapsed_time, 'seconds')


Random Forest confusion matrix:
[[3454  298]
 [ 296 3952]]
Accuracy of Random Forest method: 92.575
Elapsed time: 15.117662191390991 seconds
